### Multilab - Cleanlab extension for multi-label multi-annotator dataset
This notebook demonstrates an approach to label quality and active learning for training classification models with multilab.

In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt

### Import dependencies and get data
Please install the dependencies specified in this [requirements.txt](Multilab/main/requirements.txt) file before running the notebook.
We load the following datafiles:

labels_multiannotator is a DataFrame that contains labels from multiple annotators for each example (different number of annotations per example)  

pred_probs are are the predicted probabilites for examples that have existing annotator labels or are the predicted probabilites for examples that do not have any annotator labels  


In [2]:
dataset = pd.read_csv("/home/ponnurus/AUDetection-lightning/data/datafiles/annotationRecords1.csv")

In [3]:
pred_probs_df = pd.read_csv("/home/ponnurus/AUDetection-lightning/logs/2023-10-13T16-49-47_four/Predcition_prob.csv")

In [4]:
labels_df=pd.read_csv("/home/ponnurus/AUDetection-lightning/logs/2023-10-13T16-49-47_four/Predcition_pred.csv")

In [5]:
#get image name from path#get image name from path
pred_probs_df['image_name'] = pred_probs_df['path'].apply(lambda x: x.split('/')[-1])

In [6]:
#get image name from path
labels_df['image_name'] = labels_df['path'].apply(lambda x: x.split('/')[-1])

In [7]:
#retaining the images that are common in dataset and predicted probabilities
df1 = dataset[dataset['image_name'].isin(pred_probs_df['image_name'])]
df1=df1.drop(columns=['comments','_id','date','end_time', 'filepath','start_time', 'time_in_seconds','no_particular_expression',
'patientID', 'smile', 'start_time', 'time_in_seconds','wrinkled_forehead', 'unclear'])

In [8]:
#renaming columns for consistency
name_map={'au10_raising_of_upper_lip':'AU10',
       'au12_pulling_at_corner_lip':'AU12', 'au20_lip_stretcher':'AU20', 'au24_lip_presser':'AU24',
       'au25_parting_lips':'AU25', 'au26_jaw_drop':'AU26', 'au27_mouth_stretch':'AU27',
       'au43_eyes_closed':'AU43', 'au4_brow_lowering':'AU4', 'au6_cheek_raising':'AU6',
       'au7_tightning_of_eyelids':'AU7', 'au9_wrinkling_of_nose':'AU9'}
df1.rename(columns=name_map, inplace=True)

In [9]:
#grouping by image_name
grouped_df = df1.groupby(['image_name','annotator'], as_index=False).first()
grouped_df

,image_name,annotator,AU10,AU12,AU20,AU24,AU25,AU26,AU27,AU43,AU4,AU6,AU7,AU9
0,2021-06-01 15-01-02_000000000519.jpg,babatundeshofolu,0,0,0,0,1,0,0,1,0,0,0,0
1,2021-06-01 15-01-02_000000000519.jpg,hannahweisman,0,0,0,0,1,0,0,1,0,0,0,0
2,2021-06-01 15-01-02_000000000519.jpg,jennifer.noa,0,0,0,0,1,0,0,1,0,0,0,0
3,2021-06-01 15-01-02_000000000519.jpg,rishika.patel@ufl.edu\health,0,0,0,0,1,0,0,1,0,0,0,0
4,2021-06-01 15-01-02_000000000525.jpg,babatundeshofolu,0,0,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128047,2022-03-30 17-14-24_000000001763.jpg,aribahali,0,0,0,0,1,0,0,0,0,0,0,0
128048,2022-03-30 17-14-24_000000001764.jpg,aribahali,0,0,0,0,0,1,0,0,0,0,0,0
128049,2022-03-30 17-14-24_000000001765.jpg,aribahali,0,0,0,0,0,1,0,0,0,0,0,0
128050,2022-03-30 17-14-24_000000001767.jpg,aribahali,0,0,0,0,1,0,0,0,0,0,0,0


In [10]:
#drop the uncommon AU's
labels_df=labels_df.drop(columns=['path','AU1','AU2', 'AU14','AU15', 'AU17','AU23'])

In [11]:
order_mapping = {value: index for index, value in enumerate(labels_df['image_name'])}
grouped_df['order'] = grouped_df['image_name'].map(order_mapping)

In [12]:
grouped_df = grouped_df.sort_values(by='order').drop(columns='order')

In [13]:
grouped_df

,image_name,annotator,AU10,AU12,AU20,AU24,AU25,AU26,AU27,AU43,AU4,AU6,AU7,AU9
79919,2022-01-29 08-54-02_000000000260.jpg,hannahweisman,0,0,0,0,1,0,0,1,0,0,0,0
79918,2022-01-29 08-54-02_000000000260.jpg,ezequielbautista,0,0,0,0,1,0,0,1,0,0,0,0
79920,2022-01-29 08-54-02_000000000260.jpg,kmaisuria,0,0,0,0,0,1,0,1,0,0,0,0
79921,2022-01-29 08-54-02_000000000260.jpg,wkratochvil,0,0,0,0,0,1,0,1,0,0,0,0
79924,2022-01-29 09-09-07_000000000184.jpg,kmaisuria,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71516,2022-01-20 08-37-38_000000000430.jpg,hannahweisman,0,0,0,0,1,0,0,1,0,0,0,0
71517,2022-01-20 08-37-38_000000000459.jpg,hannahweisman,0,0,0,0,0,0,0,1,0,0,0,0
71518,2022-01-20 08-37-38_000000000460.jpg,hannahweisman,0,0,0,0,1,0,0,1,0,0,0,0
71519,2022-01-20 08-37-38_000000000462.jpg,hannahweisman,0,0,0,0,1,0,0,1,0,0,0,0


In [14]:
df = grouped_df
annotators = df['annotator'].unique()
labels = df.columns[2:]

In [15]:
AU=['AU4', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU20', 'AU24', 'AU25', 'AU26','AU27', 'AU43'] #replace with the labels in your dataset

In [16]:
df

,image_name,annotator,AU10,AU12,AU20,AU24,AU25,AU26,AU27,AU43,AU4,AU6,AU7,AU9
79919,2022-01-29 08-54-02_000000000260.jpg,hannahweisman,0,0,0,0,1,0,0,1,0,0,0,0
79918,2022-01-29 08-54-02_000000000260.jpg,ezequielbautista,0,0,0,0,1,0,0,1,0,0,0,0
79920,2022-01-29 08-54-02_000000000260.jpg,kmaisuria,0,0,0,0,0,1,0,1,0,0,0,0
79921,2022-01-29 08-54-02_000000000260.jpg,wkratochvil,0,0,0,0,0,1,0,1,0,0,0,0
79924,2022-01-29 09-09-07_000000000184.jpg,kmaisuria,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71516,2022-01-20 08-37-38_000000000430.jpg,hannahweisman,0,0,0,0,1,0,0,1,0,0,0,0
71517,2022-01-20 08-37-38_000000000459.jpg,hannahweisman,0,0,0,0,0,0,0,1,0,0,0,0
71518,2022-01-20 08-37-38_000000000460.jpg,hannahweisman,0,0,0,0,1,0,0,1,0,0,0,0
71519,2022-01-20 08-37-38_000000000462.jpg,hannahweisman,0,0,0,0,1,0,0,1,0,0,0,0


In [17]:
#converting the dataframe into a array of multiple given labels for each label per example with shape (N, M,K)
annotators = df['annotator'].unique()
indices = df.columns.difference(['image_name', 'annotator'])
classes = indices.reindex(AU)[0]  
num_samples = len(df['image_name'].unique())
num_annotators = len(annotators)
num_classes = len(classes)
labels_multiannotator = np.full((num_samples, num_annotators, num_classes), np.nan)

for i, row in df.iterrows():
    sample_idx = np.where(df['image_name'].unique() == row['image_name'])[0][0]
    annotator_idx = np.where(annotators == row['annotator'])[0][0]
    class_indices = [np.where(classes == col)[0][0] for col in classes]
    labels_multiannotator[sample_idx, annotator_idx, class_indices] = row[classes].values

In [18]:
pred_prob = pred_probs_df[pred_probs_df['image_name'].isin(df['image_name'])]

In [19]:
pred_prob=pred_prob.drop_duplicates(subset=['image_name'])

In [20]:
pred_prob=pred_prob.drop(columns=['path','AU1','AU2', 'AU14','AU15', 'AU17','AU23'])

In [21]:
pred_prob=pred_prob.drop(columns=['image_name'])

In [22]:
pred_probs=pred_prob.iloc[:, 0:].values

### Checking format and shape of parameters
 labels_multiannotator : 3D pandas DataFrame or array of multiple given labels per class for each example with shape (N, M, K)  
   
 N is the number of examples, M is the number of annotators. labels_multiannotator[n][m][k] - label for n-th example given by m-th annotator for k-th class.  
 
 For a dataset with K classes, each given label must be binary either 0(absent), 1(present) or NaN if this annotator did not label a particular example.  
        
pred_probs : np.ndarray
        An array of shape (N, K) of predicted class probabilities from a trained classifier model.

In [23]:
labels_multiannotator[0]

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]])

In [24]:
labels_multiannotator.shape #shape is (N,M,K)

(54714, 9, 12)

In [25]:
pred_probs[0]

array([0.00962529, 0.00341886, 0.00177602, 0.00043265, 0.00053205,
       0.00328887, 0.00169592, 0.0208411 , 0.02978184, 0.01950531,
       0.20444597, 0.01785214])

In [26]:
pred_probs.shape #shape is (N,K)

(54714, 12)

### Import and run code

In [32]:
import sys
sys.path.append('/home/ponnurus/AUDetection-lightning/Multilab/main')

In [33]:
import main
from main import multiannotators

In [34]:
from main.multiannotators import get_label_quality_multiannotator

In [35]:
import imp
imp.reload(multiannotators)

/tmp/ipykernel_68120/3554794139.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


<module 'main.multiannotators' from '/home/ponnurus/AUDetection-lightning/Multilab/main/multiannotators.py'>

In [36]:
results=get_label_quality_multiannotator(labels_multiannotator,pred_probs,consensus_method = "majority_vote",quality_method = "crowdlab",verbose=False)

In [37]:
results["label_quality"]

,num_annotations,consensus_label,annotator_agreement,consensus_quality_score
0,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, ...",0.180456
1,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.336268
2,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 0.75,...",0.787189
3,4,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 0.75,...",0.274303
4,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, ...",0.947935
...,...,...,...,...
54709,1,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.272093
54710,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.946003
54711,1,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.170842
54712,1,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.243381


In [38]:
results["detailed_label_quality"]

,Quality_wkratochvil,Quality_kmaisuria,Quality_hannahweisman,Quality_ezequielbautista,Quality_aribahali,Quality_kaelynnrodriguez,Quality_babatundeshofolu,Quality_rishika.patel@ufl.edu\health,Quality_jennifer.noa
0,0.052344,0.052344,0.050640,0.050640,NaN,NaN,NaN,NaN,NaN
1,0.336268,0.336268,0.336268,0.336268,NaN,NaN,NaN,NaN,NaN
2,0.787189,0.163040,0.039551,0.787189,NaN,NaN,NaN,NaN,NaN
3,0.072888,0.040913,0.055501,0.274303,NaN,NaN,NaN,NaN,NaN
4,0.202822,0.190673,0.202822,0.190673,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
54709,0.272093,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54710,0.946003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54711,0.170842,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54712,0.243381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
results["annotator_stats"]

,annotator_quality,agreement_with_consensus,worst_class,num_of_examples_labeled
ezequielbautista,0.787269,0.788483,AU4,1204
hannahweisman,0.860498,0.861731,AU7,3269
kaelynnrodriguez,0.875700,0.877003,AU9,9923
aribahali,0.904089,0.914639,AU4,17714
babatundeshofolu,0.920312,0.921352,AU26,20632
rishika.patel@ufl.edu\health,0.921428,0.922639,AU7,5930
wkratochvil,0.940861,0.961983,AU25,45039
jennifer.noa,0.953036,0.954126,AU25,8160
kmaisuria,0.955857,0.956904,AU25,16181


### Obtaining active learning score

We get the active learning scores for each datapoint (both labeled and unlabeled) by using a combination of the annotators' agremeent and model confidence. These scores represent how confident we are about an example's true label based on the currently obtained annotations; examples with the lowest scores are those for which additional labels should be collected (i.e. likely the most informative). These scores are estimated via an ActiveLab algorithm developed by the Cleanlab team, and may sometimes prioritize an already-labeled example over an unlabeled example if the annotations for the labeled example are deemed unreliable (ActiveLab appropriately estimates the value of collecting new annotations for unlabeled data vs already-labeled data).

Similar to above, the labels_multiannotator DataFrame here should only include examples that have received at least one annotation (labels_multiannotator should have the same number of rows as pred_probs). This method returns one of the two dataframes: active_learning_scores represents the scores for examples with existing annotations, and active_learning_scores_unlabeled represents the scores for examples with no annotations so far.

In [40]:
from main.multiannotators import get_active_learning_scores

In [41]:
active_learning_scores_labelled, relabelling_labeled = get_active_learning_scores(labels_multiannotator,pred_probs, None)

In [42]:
active_learning_scores_labelled

,AU4,AU6,AU7,AU9,AU10,AU12,AU20,AU24,AU25,AU26,AU27,AU43
0,0.519580,0.548522,0.495575,0.509654,0.551781,0.537802,0.550546,0.538671,0.299810,0.322456,0.767808,0.543154
1,0.864516,0.874573,0.872017,0.865731,0.876600,0.872532,0.875775,0.876308,0.863064,0.867215,0.876135,0.910917
2,0.596135,0.626061,0.383620,0.597153,0.632132,0.619594,0.629500,0.608505,0.486488,0.600643,0.630771,0.653729
3,0.767914,0.767598,0.535900,0.747489,0.768795,0.761924,0.768356,0.786606,0.725832,0.643132,0.769781,0.639591
4,0.768816,0.772196,0.742951,0.750274,0.770608,0.764073,0.771288,0.963307,0.535789,0.533116,0.770838,0.793282
...,...,...,...,...,...,...,...,...,...,...,...,...
54709,0.552510,0.585336,0.538863,0.553118,0.593388,0.575773,0.590888,0.659516,0.567686,0.568670,0.589610,0.799007
54710,0.936110,0.940816,0.934235,0.936290,0.941740,0.939504,0.941407,0.938653,0.935161,0.937700,0.941963,0.945320
54711,0.541987,0.577510,0.529697,0.544099,0.583361,0.567428,0.580758,0.573112,0.529227,0.561484,0.585800,0.626404
54712,0.569761,0.601111,0.556915,0.570728,0.607957,0.592374,0.605644,0.574281,0.573735,0.574840,0.604754,0.620572


##### Images required to be re-labelled based on the criteria and threshold set in mutliannotators.get_active_learning_scores

In [43]:
relabelling_labeled

,Index,ClassLabel,ActiveLearningScore,QualityOfConsensusLabeled,AnnotatorLabels,CL,Pred,ConfidenceScore,NormalizedSumAnnotatorWeights
116,9,AU25,0.404754,0.183892,"[1.0, 0.0, 0.0, 1.0, nan, nan, nan, nan, nan]",0,0.375474,0.249052,0.499374
367,30,AU24,0.491299,0.490865,"[0.0, 0.0, 1.0, 1.0, nan, nan, nan, nan, nan]",0,0.041815,0.916370,0.491825
1310,109,AU7,0.361138,0.270015,"[0.0, 1.0, 0.0, 1.0, nan, nan, nan, nan, nan]",0,0.001957,0.996085,0.490462
1354,112,AU27,0.322025,0.170738,"[1.0, 1.0, 0.0, 0.0, nan, nan, nan, nan, nan]",0,0.001029,0.997942,0.488135
1622,135,AU7,0.454025,0.427458,"[0.0, 1.0, 0.0, 1.0, nan, nan, nan, nan, nan]",0,0.019449,0.961102,0.490462
...,...,...,...,...,...,...,...,...,...
603692,50307,AU25,0.497283,0.556198,"[0.0, nan, nan, nan, nan, nan, nan, nan, 1.0]",1,0.500931,0.001861,0.493324
648611,54050,AU43,0.423832,0.144165,"[0.0, nan, nan, nan, nan, nan, nan, 1.0, nan]",1,0.580684,0.161367,0.493435
648619,54051,AU24,0.454210,0.443811,"[1.0, nan, nan, nan, nan, nan, nan, 0.0, nan]",0,0.100110,0.799781,0.465290
648620,54051,AU25,0.469684,0.443811,"[0.0, nan, nan, nan, nan, nan, nan, 1.0, nan]",0,0.250043,0.499913,0.490253


#### Unlabelled data

In [44]:
active_learning_scores_unlabeled, relabelling_unlabeled = get_active_learning_scores(None, None,pred_probs)

In [45]:
active_learning_scores_unlabeled

,AU4,AU6,AU7,AU9,AU10,AU12,AU20,AU24,AU25,AU26,AU27,AU43
0,0.719268,0.739084,0.744329,0.748619,0.748301,0.739499,0.744585,0.683457,0.654910,0.687722,0.402773,0.693000
1,0.747793,0.749556,0.656997,0.742814,0.748914,0.742030,0.749450,0.659129,0.694808,0.738460,0.746128,0.476079
2,0.746920,0.748912,0.726731,0.749716,0.748197,0.742499,0.749236,0.717993,0.699669,0.748072,0.745344,0.623288
3,0.654832,0.737773,0.746169,0.747629,0.749232,0.738874,0.744588,0.593653,0.268834,0.638350,0.737844,0.642222
4,0.657253,0.723768,0.738506,0.742263,0.747849,0.736167,0.738024,0.263827,0.610186,0.738488,0.739959,0.591363
...,...,...,...,...,...,...,...,...,...,...,...,...
54709,0.745161,0.748186,0.748972,0.749357,0.744003,0.748101,0.743921,0.530949,0.646792,0.717325,0.747141,0.369907
54710,0.747293,0.749775,0.749778,0.749727,0.749735,0.748743,0.749213,0.708060,0.698873,0.732142,0.738791,0.622129
54711,0.749703,0.747313,0.749213,0.749952,0.749254,0.748091,0.749315,0.679901,0.715829,0.714118,0.736177,0.588865
54712,0.745949,0.749493,0.749016,0.749075,0.747747,0.748160,0.747421,0.740406,0.672025,0.746341,0.749726,0.645357


In [46]:
relabelling_unlabeled

,Row_Index,AU,Predicted_Probability
0,0,AU27,0.402773
1,1,AU43,0.476079
2,3,AU25,0.268834
3,4,AU24,0.263827
4,6,AU43,0.354229
...,...,...,...
60356,54704,AU24,0.304884
60357,54705,AU43,0.491076
60358,54706,AU25,0.399530
60359,54706,AU43,0.336362


### Results
Lastly, we can obtain relevant findings of the examples by their active learning scores, and obtain the index of the examples with the lowest scores or by setting a threshold; these are the least confident examples which we will want to collect more labels for.


In [47]:
image_names = df['image_name'].unique()

In [48]:
relabelling_labeled['Image_Name']=image_names[relabelling_labeled['Index']]

In [49]:
relabelling_labeled

,Index,ClassLabel,ActiveLearningScore,QualityOfConsensusLabeled,AnnotatorLabels,CL,Pred,ConfidenceScore,NormalizedSumAnnotatorWeights,Image_Name
116,9,AU25,0.404754,0.183892,"[1.0, 0.0, 0.0, 1.0, nan, nan, nan, nan, nan]",0,0.375474,0.249052,0.499374,2022-01-29 09-24-12_000000000305.jpg
367,30,AU24,0.491299,0.490865,"[0.0, 0.0, 1.0, 1.0, nan, nan, nan, nan, nan]",0,0.041815,0.916370,0.491825,2022-01-29 09-54-22_000000000006.jpg
1310,109,AU7,0.361138,0.270015,"[0.0, 1.0, 0.0, 1.0, nan, nan, nan, nan, nan]",0,0.001957,0.996085,0.490462,2022-01-27 11-14-42_000000000248.jpg
1354,112,AU27,0.322025,0.170738,"[1.0, 1.0, 0.0, 0.0, nan, nan, nan, nan, nan]",0,0.001029,0.997942,0.488135,2022-01-27 11-14-42_000000000275.jpg
1622,135,AU7,0.454025,0.427458,"[0.0, 1.0, 0.0, 1.0, nan, nan, nan, nan, nan]",0,0.019449,0.961102,0.490462,2022-01-27 11-14-42_000000000305.jpg
...,...,...,...,...,...,...,...,...,...,...
603692,50307,AU25,0.497283,0.556198,"[0.0, nan, nan, nan, nan, nan, nan, nan, 1.0]",1,0.500931,0.001861,0.493324,2021-11-20 05-32-41_000000000255.jpg
648611,54050,AU43,0.423832,0.144165,"[0.0, nan, nan, nan, nan, nan, nan, 1.0, nan]",1,0.580684,0.161367,0.493435,2021-12-11 10-08-36_000000000144.jpg
648619,54051,AU24,0.454210,0.443811,"[1.0, nan, nan, nan, nan, nan, nan, 0.0, nan]",0,0.100110,0.799781,0.465290,2021-12-11 10-08-36_000000000145.jpg
648620,54051,AU25,0.469684,0.443811,"[0.0, nan, nan, nan, nan, nan, nan, 1.0, nan]",0,0.250043,0.499913,0.490253,2021-12-11 10-08-36_000000000145.jpg


In [50]:
relabelling_unlabeled['Image_Name']=image_names[relabelling_unlabeled['Row_Index']]

In [51]:
relabelling_unlabeled

,Row_Index,AU,Predicted_Probability,Image_Name
0,0,AU27,0.402773,2022-01-29 08-54-02_000000000260.jpg
1,1,AU43,0.476079,2022-01-29 09-09-07_000000000184.jpg
2,3,AU25,0.268834,2022-01-29 09-09-07_000000000749.jpg
3,4,AU24,0.263827,2022-01-29 09-24-12_000000000072.jpg
4,6,AU43,0.354229,2022-01-29 09-24-12_000000000223.jpg
...,...,...,...,...
60356,54704,AU24,0.304884,2022-01-20 08-37-38_000000000107.jpg
60357,54705,AU43,0.491076,2022-01-20 08-37-38_000000000108.jpg
60358,54706,AU25,0.399530,2022-01-20 08-37-38_000000000109.jpg
60359,54706,AU43,0.336362,2022-01-20 08-37-38_000000000109.jpg
